## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tqdm import tqdm

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path_en = './opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-train.en'
path_uk = './opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-train.uk'

In [3]:
en_sents = open(path_en).read().splitlines()
uk_sents = open(path_uk).read().replace('\xa0', ' ').splitlines()

In [4]:
(en_sents[0], uk_sents[0]), (en_sents[1], uk_sents[1])

(('Consistant with our other victims.', 'Как и все остальные жертвы мясника.'),
 ('The children.', 'Діти.'))

In [7]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=[path_en], trainer=trainer_en)

tokenizer_uk = Tokenizer(BPE())
tokenizer_uk.pre_tokenizer = Whitespace()
trainer_uk = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_uk.train(files=[path_uk], trainer=trainer_uk)

In [8]:
tokenizer_en.save('tokenizer_en')
tokenizer_uk.save('tokenizer_uk')

In [168]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_uk = Tokenizer.from_file("tokenizer_uk")

In [169]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [170]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_uk.token_to_id('[PAD]')
PAD_IDX

3

In [8]:
max_len_en, max_len_uk = 50, 50

In [15]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_uk = [encode(t, tokenizer_uk, max_len_uk) for t in uk_sents]

In [18]:
np.array(X_en).shape, np.array(X_uk).shape

/tmp/ipykernel_1865325/1698114856.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(X_en).shape, np.array(X_uk).shape


((1000000,), (1000000,))

In [9]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_uk):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)
        
        self.texts_uk = [torch.LongTensor(sent) for sent in texts_uk]
        self.texts_uk = torch.nn.utils.rnn.pad_sequence(self.texts_uk, padding_value=PAD_IDX)

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_uk = self.texts_uk[:, index]

        return ids_en, ids_uk

In [ ]:
X_en_train, X_en_valid, X_uk_train, X_uk_valid = train_test_split(X_en, X_uk, test_size=0.05)

In [24]:
training_set = Dataset(X_en_train, X_uk_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True)

valid_set = Dataset(X_en_valid, X_uk_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

In [10]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

DEVICE = torch.device('cpu' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [11]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_ru) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) # чтобы батч был в конце
        texts_ru = texts_ru.T.to(DEVICE) # чтобы батч был в конце
        
        # помимо текста в модель еще нужно передать целевую последовательность
        # но не полную а без 1 последнего элемента
        # а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
        texts_ru_input = texts_ru[:-1, :]
        
        
        # в трансформерах нет циклов как в лстм 
        # каждый элемент связан с каждым через аттеншен
        # чтобы имитировать последовательную обработку
        # и чтобы не считать аттеншн с паддингом 
        # в трансформерах нужно считать много масок
        # подробнее про это по ссылкам выше
        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)
        logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()
        
        # сравниваем выход из модели с целевой последовательностью уже с этим последним элементом
        texts_ru_out = texts_ru[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []
    
    model.eval()  
    with torch.no_grad():
        for i, (texts_en, texts_ru) in enumerate(iterator):
            texts_en = texts_en.T.to(DEVICE)
            texts_ru = texts_ru.T.to(DEVICE)

            texts_ru_input = texts_ru[:-1, :]

            (texts_en_mask, texts_ru_mask, 
            texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)

            logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                           texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

            
            texts_ru_out = texts_ru[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
            epoch_loss.append(loss.item())
            
    return np.mean(epoch_loss)

In [12]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
UK_VOCAB_SIZE = tokenizer_uk.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, UK_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [31]:
torch.cuda.empty_cache()

from timeit import default_timer as timer
NUM_EPOCHS = 20

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

Loss: 7.4738642024993895;
Loss: 6.882545293807984;
Loss: 6.578410665512085;
Loss: 6.37068803691864;
Loss: 6.210313975334167;
Loss: 6.080472269852956;
Loss: 5.968622493062701;
Loss: 5.871277550458908;
Loss: 5.7835335768593685;
First epoch - 4.869278980255127, saving model..
Epoch: 1, Train loss: 5.744, Val loss: 4.869,            Epoch time=445.096s
Loss: 4.917781402587891;
Loss: 4.884084643363953;
Loss: 4.847352036794026;
Loss: 4.814586849451065;
Loss: 4.781755893135071;
Loss: 4.750508355140686;
Loss: 4.719063627924238;
Loss: 4.6903286854028705;
Loss: 4.661846391465929;
Improved from 4.869278980255127 to 4.219856056213379, saving model..
Epoch: 2, Train loss: 4.647, Val loss: 4.220,            Epoch time=440.598s
Loss: 4.296400611400604;
Loss: 4.272436888694763;
Loss: 4.253441939671834;
Loss: 4.235538808345795;
Loss: 4.217567175769806;
Loss: 4.2003291554450986;
Loss: 4.182093508516039;
Loss: 4.162939817488193;
Loss: 4.146344013002183;
Improved from 4.219856056213379 to 3.81545216274261

In [13]:
transformer

Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_feature

In [14]:
transformer = torch.load('model').to(DEVICE)

In [16]:
def translate(text):
    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_uk.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_uk_mask, 
    texts_en_padding_mask, texts_uk_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_uk_mask,
                   texts_en_padding_mask, texts_uk_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_uk.token_to_id('[SEP]'), tokenizer_uk.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_uk_mask, 
        texts_en_padding_mask, texts_uk_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_uk_mask,
                       texts_en_padding_mask, texts_uk_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_uk.id_to_token(i).replace('##', '') for i in output_ids[1:]]))

На всех тестировать я все же не стал, особо смысла мне кажется не имеет, но результат хороший в любом случае:

In [17]:
for sent in en_sents[:100]:
    print('English: ', sent)
    print('Ukrainian: ', translate(sent))
    print( )

English:  Consistant with our other victims.
Ukrainian:  И все равно , что с нашими другими жерт вами .

English:  The children.
Ukrainian:  Діти .

English:  I still have some friends in high places.
Ukrainian:  У меня еще есть друзья в высо ких местах .

English:  Anything else you can tell us?
Ukrainian:  Что - нибудь ещё ты можешь рассказать нам ?

English:  Successfully moved to trash.
Ukrainian:  Успі шно пересунуто до смітника .

English:  The given name could not be resolved to a unique server. Make sure your network is setup without any name conflicts between names used by Windows and by UNIX name resolution.
Ukrainian:  Не вдалося визначити назву , який буде використано уніка льну сервер . Переконайтеся , що вашу мережу не буде використано для будь - якого з назв конфліктів , які використовуються у Windows і UNIX .

English:  Jake, your mom's on the phone!
Ukrainian:  Джейк , твоя мати на телефо ні !

English:  Father, no!
Ukrainian:  Батьку , ні !

English:  Data tool
Ukrain

Ukrainian:  Konqueror є разом з веб - і переглядача файлів .

English:  Delete the selected task.
Ukrainian:  Вилучити позначений завдання .

English:  %1%
Ukrainian:  % 1 %

English:  - Mm-hmm.
Ukrainian:  - Угу .

English:  (sighs) I just...
Ukrainian:  Я просто ...



Сделаем функцию для генерации батчами:

In [411]:
def translate_texts(list_of_texts):
    gen_texts = {}
    input_ids = [[tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')] for text in list_of_texts]
    output_ids = [[tokenizer_uk.token_to_id('[CLS]')] for _ in range(len(list_of_texts))]

    input_ids = [torch.LongTensor(ids) for ids in input_ids]
    output_ids = [torch.LongTensor(ids) for ids in output_ids]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence(input_ids)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence(output_ids)

    (texts_en_mask, texts_ru_mask, texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

    pred = logits.argmax(2)
    output_ids_pad = torch.cat([output_ids_pad, pred])

    for _ in tqdm(range(20)):
        (texts_en_mask, texts_uk_mask, texts_en_padding_mask, texts_uk_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_uk_mask,
                                   texts_en_padding_mask, texts_uk_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].unsqueeze(0)

        if tokenizer_uk.token_to_id('[SEP]') in pred or tokenizer_uk.token_to_id('[PAD]') in pred:
            pr = pred.squeeze(0)
            leave_ids = np.where((pr == tokenizer_uk.token_to_id('[SEP]')) | (pr == tokenizer_uk.token_to_id('[PAD]')))[0]
            leave_output_ids = output_ids_pad[:, [id_ for id_ in range(output_ids_pad.shape[1]) if id_ in leave_ids]]

            for i, id_ in enumerate(leave_ids):
                gen_text = ' '.join([tokenizer_uk.id_to_token(j) for j in leave_output_ids[:, i]])
                gen_texts[id_] = gen_text

            input_ids_pad = input_ids_pad[:, [id_ for id_ in range(input_ids_pad.shape[1]) if id_ not in leave_ids]]
            output_ids_pad = output_ids_pad[:, [id_ for id_ in range(output_ids_pad.shape[1]) if id_ not in leave_ids]]
            pred = pred[:, [id_ for id_ in range(pred.shape[1]) if id_ not in leave_ids]]
        output_ids_pad = torch.cat([output_ids_pad, pred])

    for i in range(output_ids_pad.shape[1]):
        if i in gen_texts.keys():
            continue
        else:
            gen_text = ' '.join([tokenizer_uk.id_to_token(j) for j in output_ids_pad[:, i]])
            gen_texts[i] = gen_text
    gen_texts = dict(sorted(gen_texts.items(), key=lambda x: x[0]))
    gen_texts = list(gen_texts.values())
    return gen_texts

In [434]:
import random

inds = random.choices(range(5, 1000), k=50)
orig_en_texts = np.array(en_sents[:1000])[inds]

translated_uk_texts = translate_texts(orig_en_texts)

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 13.06it/s]


In [435]:
for en_sent, uk_sent in zip(orig_en_texts, translated_uk_texts):
    print('English: ', en_sent)
    print('Ukrainian: ', uk_sent)
    print( )

English:  Hook me up?
Ukrainian:  [CLS] У ко пу шки , у нас є можливість , що у вас є пу шки , у грі є ко

English:  - You busy?
Ukrainian:  [CLS] У нас є повно му , що ви знайдете у грі , у грі є ко пу ків , що ви

English:  You got them going out there, Mike.
Ukrainian:  [CLS] У вас є їх там , Майк , у нас є всі єю програмою , що у вас є повно му

English:  It's your father.
Ukrainian:  [CLS] У нас є ваш батько , і у грі є ко пу ків , що гри ми , вті хи ,

English:  The SERIESSUM() function returns the sum of a power series.
Ukrainian:  [CLS] Функція S ER I ES SU M () повертає суму з використанням ряд ження м , що програма для гри у

English:  This dialog will allow you to specify a PostgreSQL account that has the necessary permissions to access the Krecipes PostgreSQL database. This account may either be a PostgreSQL superuser or have the ability to both create new PostgreSQL users and databases. If no superuser or privileged account is given, the account'postgres' will be attempted

В общем почти все ок, за исключением того, что [SEP] токен почему-то генерируется не во всех случаях.


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

Данная техника используется для обучения модели машинного перевода в том случае, если у нас мало данных таргетного языка, того на который мы хотим переводить. <br>

Например, у нас есть параллельный корпус en-ru в 10000 предложений, и дополнительный корпус en в 100000 предложений, и мы хотим обучить модель машинного перевода ru-en. Мы берем и обучаем модель машиннного перевода с en на ru на 10000 предложениях. После чего генерируем с помощью данной модели перевод 100000 предложений en из дополнительного корпуса. После чего складываем наш исходных параллельный корпус со сгенерированным корпусом 10000 + 100000 = 110000. После чего уже обучаем еще одну модель для перевода с ru на en. Авторы статьи пишут, что такая модель показывает 2/3 результативности модели, обученной в обычном режиме, без back translation.